In [ ]:
import os
import ee
import pandas as pd
import geopandas as gp
import numpy as np
import matplotlib.pyplot as plt

import rsfuncs as rs

ee.Initialize()
%load_ext autoreload
%autoreload 2

In [ ]:
shp = gp.read_file("../shape/cv.shp")

In [ ]:
area = rs.gdf_to_ee_poly(shp)
polys = rs.gen_polys(area)

In [ ]:
data = rs.load_data()

In [ ]:
def img_to_arr(eeImage, var_name):
    temp = eeImage.select(var_name).clip(area)
    latlon = eeImage.pixelLonLat().addBands(temp)
    
    latlon = latlon.reduceRegion(
        reducer = ee.Reducer.toList(),
        geometry = area, 
        scale = 1000
        )
    
    data = np.array((ee.Array(latlon.get(var_name)).getInfo()))
    lats = np.array((ee.Array(latlon.get('latitude')).getInfo()))
    lons = np.array((ee.Array(latlon.get('longitude')).getInfo()))
    
    lc,freq = np.unique(data,return_counts = True)
    
    return data, lats,lons 

def imc_to_arr(eeImage):
    temp = eeImage.filterBounds(area).first().pixelLonLat()
    
    latlon = temp.reduceRegion(
        reducer = ee.Reducer.toList(),
        geometry = area, 
        scale = 1000
        )
    
    data = np.array((ee.Array(latlon.get('cropland')).getInfo()))
    lats = np.array((ee.Array(latlon.get('latitude')).getInfo()))
    lons = np.array((ee.Array(latlon.get('longitude')).getInfo()))
    
    lc,freq = np.unique(data,return_counts = True)
    
    return data, lats,lons 

def arr_to_img(data,lats,lons):
    uniquelats = np.unique(lats)
    uniquelons = np.unique(lons)
    
    ncols = len(uniquelons)
    nrows = len(uniquelats)
    
    ys = uniquelats[1] - uniquelats[0]
    xs = uniquelons[1] - uniquelons[0]
    
    arr = np.zeros([nrows, ncols], np.float32)
    
    counter = 0
    for y in range(0, len(arr),1):
        for x in range(0, len(arr[0]),1):
            if lats[counter] == uniquelats[y] and lons[counter] == uniquelons[x] and counter < len(lats)-1:
                counter+=1
                arr[len(uniquelats)-1-y,x] = data[counter]
                
    return arr

def freq_hist(eeImage, area, var_name):    
    freq_dict = ee.Dictionary(
      eeImage.reduceRegion(ee.Reducer.frequencyHistogram(), area, 30).get(var_name)
    );
    
    return freq_dict


def gen_polys(geometry, dx=0.5, dy=0.5):
    
    '''
    Return ee.ImaceCollection of polygons used to submit full res (30m landsat; 10m sentinel) resolution
    '''
    
    bounds = ee.Geometry(geometry).bounds()
    coords = ee.List(bounds.coordinates().get(0))
    ll = ee.List(coords.get(0))
    ur = ee.List(coords.get(2))
    xmin = ll.get(0)
    xmax = ur.get(0)
    ymin = ll.get(1)
    ymax = ur.get(1)

    latlist = ee.List.sequence(ymin, ymax, dx)
    lonlist = ee.List.sequence(xmin, xmax, dy)
    
    polys = []
    
    for lon in lonlist.getInfo():
        for lat in latlist.getInfo():
        
            def make_rect(lat, lon):
                lattemp = ee.Number(lat)
                lontemp = ee.Number(lon)
                uplattemp = lattemp.add(dy)
                lowlontemp = lontemp.add(dx)

                return ee.Feature(ee.Geometry.Polygon([[lontemp, lattemp],[lowlontemp, lattemp],[lowlontemp, uplattemp],[lontemp, uplattemp]]))
            
            poly = make_rect(lat,lon)
            polys.append(poly)
    
    return ee.FeatureCollection(ee.List(polys))

In [ ]:
def main(ee_image, area):
    cdl = ee.Image(ee_image)
    polys = gen_polys(area)
    d = polys.getInfo()
    
    results = []

    for i in d['features']:
        aoi = ee.Geometry.Polygon(i['geometry']['coordinates']).intersection(area)
        t = cdl.reduceRegion(ee.Reducer.frequencyHistogram(), aoi, 30).get('cropland').getInfo()
        if not t:
            continue
        else:
            results.append(t)
    
    allcrops = []

    for i in results:
        allcrops.append(list(i.keys()))
        
    temp = [item for sublist in allcrops for item in sublist]
    fin = dict.fromkeys(set(temp),0)
    
    for i in results:
        for k,v in i.items():
            fin[k] += i[k]
            
    return fin

In [ ]:
cols = ["USDA/NASS/CDL/2008", "USDA/NASS/CDL/2009", 
        "USDA/NASS/CDL/2010", "USDA/NASS/CDL/2011",
        "USDA/NASS/CDL/2012", "USDA/NASS/CDL/2013",
        "USDA/NASS/CDL/2014", "USDA/NASS/CDL/2015",
        "USDA/NASS/CDL/2016", "USDA/NASS/CDL/2017",
        "USDA/NASS/CDL/2018"]

In [ ]:
yearly = []

for i in cols:
    print(i)
    yearly.append(main(i,area))
    

In [ ]:
fractions = []

for i in yearly:
    annual_sum = sum(i.values())
    newdict  = {k: v / annual_sum for k,v in i.items()}
    fractions.append(newdict)

In [ ]:
# Figure out the percent of tree crops over time

pct_treecrops = []
pct_fallow = []
pct_doublecrop = []
pct_row = []
pct_truck = []
pct_field = []
pct_native = []

for i in fractions:
    treecrops = ["75", "75", "76","66","77", "223", "68", "210", "220", "67", "70", "71", "204", "211"] 
    fallow = ["59","61"]
    doublecropped = ["225", "226", "227", "229", "231", "232", "233", "234", "235", "236", "237", "238"]
    rowcrops = ["1", "5", "6", "12", "13"]
    trucks = ["242", "243", "244", "245", "246", "247", "248", "249", "250", "55", "214", "216","219","221", "227"]
    fieldcrops = ["10",  "14", "224", "33", "34", "35", "36", "38", "39", "41", "42", "43", "46", "47", "48" ,
      "49", "50", "51", "52", "53", "54",  "56", "57","206","207", "208", "209","222", "229"]
    native = ["176","63","64", "65", "131","58", "141", "142", "143", "152"]
    
    annual_pct_native = 0
    annual_pct_double = 0
    annual_pct_tree = 0
    annual_pct_fallow = 0
    annual_pct_row = 0
    annual_pct_trucks = 0
    annual_pct_field = 0

    for k,v in i.items():
        if k in treecrops:
            annual_pct_tree += i[k]
        if k in fallow:
            annual_pct_fallow += i[k]
        if k in doublecropped:
            annual_pct_double += i[k]
        if k in rowcrops:
            annual_pct_row+=i[k]
        if k in trucks:
            annual_pct_trucks+=i[k]
        if k in fieldcrops:
            annual_pct_field+=i[k]
        if k in native:
            annual_pct_native+=i[k]
            
    pct_treecrops.append(annual_pct_tree)
    pct_fallow.append(annual_pct_fallow)
    pct_doublecrop.append(annual_pct_double)
    pct_row.append(annual_pct_row)
    pct_field.append(annual_pct_field)
    pct_truck.append(annual_pct_trucks)
    pct_native.append(annual_pct_native)


plt.figure(figsize = (10,8))
plt.title("Annual change in CV Crops")
plt.plot(range(2008,2019),pct_treecrops, label = "pct tree crops")
plt.plot(range(2008,2019),pct_fallow, label = "pct fallow")
plt.plot(range(2008,2019),pct_doublecrop, label = "pct doublecropped")
plt.plot(range(2008,2019),pct_row, label = "pct rowcrops")
plt.plot(range(2008,2019),pct_field, label = "pct field crops")
plt.plot(range(2008,2019),pct_native, label = "pct Native veg")

plt.legend()
plt.show()

In [ ]:
allcrops = []

for i in fractions:
    allcrops.append(list(i.keys()))

ac = [item for sublist in allcrops for item in sublist]
t = set(ac)
t2 = [int(x) for x in t]
print(sorted(t2))
print(len(t2))
# These are all the 109 crop types in the CV over all the years 

In [ ]:
cdl2f = {
# Water = water(83), wetlands(87), Aquaculture(92), Open Water(111), Perreniel Ice / Snow (112)
1 : ["83", "87", "92", "111", "112"], 
# Urban = developed high intensity(124), developed medium intensity(123)
2 : ["124", "123"], 
# Native = grassland/pasture(176), Forest(63), Shrubs(64), barren(65, 131), Clover/Wildflowers(58)
# Forests (141 - 143), Shrubland (152)
3 : ["176","63","64", "65", "131","58", "141", "142", "143", "152"], 
# Orchards, groves, vineyards = 
4 : [""],
# Pasture / hay = other hay / non alfalfa (37)
5 : ["37"],
# Row Crops = corn (1), soybeans (5),Sunflower(6) sweet corn (12), pop corn (13), double winter/corn (225), 
# double oats/corn(226), double barley/corn(237), double corn / soybeans
6 : ["1", "5", "6", "12", "13", "225", "226", "237", "239"] ,
# Small Grains = Spring wheat (23), winter wheat (24), other small grains (25), winter wheat / soybeans (26), 
# rye (27), oats (28), Millet(29), dbl soybeans/oats(240)
7 : ["23", "24", "25", "26", "27", "28", "29", "240"] ,
# Idle/fallow = Sod/Grass Seed (59), Fallow/Idle Cropland(61), 
8 : ["59","61"],
# Truck, nursery, and berry crops = 
# Blueberries (242), Cabbage(243), Cauliflower(244), celery (245), radishes (246), Turnips(247)
# Eggplants (249), Cranberries (250), Caneberries (55), Brocolli (214), Peppers(216), 
# Greens(219), Strawberries (221), Lettuce (227), Double Lettuce/Grain (230 - 233)
9 : ["242", "243", "244", "245", "246", "247", "248", "249", "250", "55", "214", "216","219","221", "227", "230", "231", "232", "233"], 

# Citrus and subtropical = Citrus(72), Oranges (212), Pommegranates(217)
10 : ["72", "212", "217"] ,

# Field Crops = 
# Peanuts(10),Mint (14),Canola (31),  Vetch(224),  Safflower(33) , RapeSeed(34), 
# Mustard(35) Alfalfa (36),Camelina (38), Buckwheat (39), Sugarbeet (41), Dry beans (42), Potaoes (43)
# Sweet potatoes(46), Misc Vegs & Fruits (47), Cucumbers(50)
# Chick Peas(51),Lentils(52),Peas(53),Tomatoes(54)Hops(56),Herbs(57),Carrots(206),
# Asparagus(207),Garlic(208), Cantaloupes(209), Honeydew Melons (213), Squash(222), Pumpkins(229), 

11 : ["10",  "14", "224", "31","33", "34", "35", "36", "38", "39", "41", "42", "43", "46", "47", "48" ,
      "49", "50", "51", "52", "53", "54",  "56", "57","206","207", "208", "209","213","222", "229"] ,
    

# Vineyards = Grapes(69)
12 : ["69"],
# Pasture = Switchgrass(60)
13 : ["60"],
# Grain and hay = Sorghum(4), barley (21), Durham wheat (22), Triticale (205), 
# Dbl grain / sorghum (234 - 236), Dbl 
14 : ["4", "21", "22", "205", "234", "235", "236"],
# livestock feedlots, diaries, poultry farms = 
15 : [""],
# Deciduous fruits and nuts = Pecans(74), Almonds(75), 
# Walnuts(76), Cherries (66), Pears(77), Apricots (223), Apples (68), Christmas Trees(70)
# Prunes (210), Plums (220), Peaches(67), Other Tree Crops (71), Pistachios(204), 
# Olives(211), Nectarines(218), Avocado (215)
    
16 : ["74", "75", "76","66","77", "223", "68", "210", "220", "67", "70", "71", "204", "211","215","218"],

# Rice = Rice(3)
17 : ["3"],
# Cotton = Cotton (2) , Dbl grain / cotton (238-239)
18 : ["2", "238", "239"], 
# Developed = Developed low intensity (122) developed open space(121)
19 : ["122", "121"],
# Cropland and Pasture
20 : [""],
# Cropland = Other crops (44)
21 : ["44"], 
# Irrigated row and field crops = Woody Wetlands (190), Herbaceous wetlands = 195
22 : ["190", "195"] 
}

In [ ]:
# Test that all crop types returned by the CDL are accounted for in the lookup table

testlist = [item for sublist in list(cdl2f.values()) for item in sublist]

for i in t2:
    if str(i) not in testlist:
        print(i)
    else:
        continue


In [ ]:
# Disaggregate the annual fractions to the FMP fractions 

fmp = {k: 0 for k in range(1, 1+len(cdl2f))}

# Loop through the annual CDL fractions dict
for i in fractions[:1]:
    for k,v in i.items():
        # For each CDL class (keys), find the corresponding FMP class in the cdl2f dict values. 
        for k2,v2 in cdl2f.items():
            # IF the CDL class is in the FMP lookup table values: add the CDL fraction to the fmp dict
            if k in v2:
                fmp[k2] += i[k]
            else:
                continue

    
# Make sure it sums to 1
sum(list(fmp.values()))

In [ ]:
t = pd.DataFrame.from_dict(fmp, orient='index')
t.columns = ['frac']

In [ ]:
# Now read the kc files, calculate the monthly weighted average
kc = pd.read_csv("../data/fmp_kc_faunt.csv")
df = pd.merge(kc,t, left_on = "Crop_id", right_index = True)

In [ ]:
multipliers = []
for i in df.columns[2:-1]:
    mult = sum(df[i] * df.frac)
    multipliers.append(mult)

In [ ]:
plt.plot(multipliers)

In [ ]:
# Disaggregate the annual fractions to the FMP fractions 

multipliers = []

# Loop through the annual CDL fractions dict
for i in fractions:
    fmp = {k: 0 for k in range(1, 1+len(cdl2f))}

    for k,v in i.items():
        # For each CDL class (keys), find the corresponding FMP class in the cdl2f dict values. 
        for k2,v2 in cdl2f.items():
            # IF the CDL class is in the FMP lookup table values: add the CDL fraction to the fmp dict
            if k in v2:
                fmp[k2] += i[k]
            else:
                continue

    # Join the FMP to the kc table
    t = pd.DataFrame.from_dict(fmp, orient='index')
    t.columns = ['frac']
    df = pd.merge(kc,t, left_on = "Crop_id", right_index = True)
    
    # Multiply the fractions by monthly kcs, take the sum across all classes, then append to multipliers list
    for i in df.columns[2:-1]:
        mult = sum(df[i] * df.frac)
        multipliers.append(mult)



In [ ]:
# Sanity Check
plt.plot(multipliers)

In [ ]:
# Save
dates = pd.date_range(start='1/1/2008', periods=132, freq='M')

mult_df = pd.DataFrame(multipliers, index = dates)
mult_df.columns = ['et_multiplier']
mult_df.to_csv("../data/et_kc.csv")